In [3]:
import  io,os
from google.cloud import vision
from google.cloud.vision import types
import glob, pickle, re
import pandas as pd


In [2]:
#océrisation
client = vision.ImageAnnotatorClient()

    try : 
        rep_pickle = rep.replace("2017","2017_pickle")
        os.mkdir(rep_pickle)
    except FileExistsError:
        pass
    for file in glob.glob(rep+"/*"):
        #print(file)
        out_pickle = file.replace("2017","2017_pickle")
        out_pickle = out_pickle.replace("jpeg","pickle")
        #print(out_pickle)

        with io.open(file, 'rb') as image_file:
            content = image_file.read()
        image = types.Image(content=content)
        response = client.document_text_detection(image=image)
        document = response.full_text_annotation
        #print(document.text)
        #print("\n")

        with open(out_pickle,'wb') as out :
            pickle.dump(document, out)      

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or
explicitly create credential and re-run the application. For more
information, please see
https://developers.google.com/accounts/docs/application-default-credentials.

In [19]:
# traitement des textes
texts_dic = {}
find_num = re.compile("2017_pickle/CFA_2017_pickle_notebook(\d)")
for rep in glob.glob("2017_pickle/*") : 
    num = (find_num.match(rep).group(1))
    texts_dic[int(num)]=[]
    for file in glob.glob(rep+'/*') : 
        pkl_file = open(file, 'rb')
        document = pickle.load(pkl_file)
        br = re.compile(r'^type: (.*)$')
        for page in document.pages:
                for block in page.blocks:
                    block_words = []
                    for paragraph in block.paragraphs:
                        block_words.extend(paragraph.words)

                    block_symbols = []
                    for word in block_words:
                        block_symbols.extend(word.symbols)

                    block_text = ''
                    for symbol in block_symbols:
                        block_text = block_text + symbol.text
                        #block_text += symbol.property.detected_break.type
                        #print(symbol.property.detected_break)
                        machin = br.match(str(symbol.property.detected_break))
                        try :
                            if (machin.group(1))=='SPACE' : 
                                block_text+=" "
                            elif ((machin.group(1)=='EOL_SURE_SPACE') or ((machin.group(1))=='LINE_BREAK')) :
                                block_text+="\n"
                        except AttributeError :
                            pass

                    if len (block_text) < 200 :
                        pass
                    else : 
                        texts_dic[int(num)].append(block_text)



In [20]:
#écriture fichier binaire
with open("textes_notebooks.pickle",'wb') as out_text :
            pickle.dump(texts_dic, out_text)

In [21]:
#lecture fichier binaire
with open("textes_notebooks.pickle",'rb') as truc :
            textes = pickle.load(truc)

In [32]:
len(textes[4])

1044

In [23]:
len(textes[1])+len(textes[2])+len(textes[3])+len(textes[4])+ len(textes[5])

3918

In [34]:
textes[2][100]

'When the policy rate is above (below) the neutral rate, the monetary policy is said to\nbe contractionary (expansionary). In general, contractionary policy is associated with\na decrease in the growth rate of money supply, while expansionary policy increases its\ngrowth rate.\n'